In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
import pandas as pd

from safe_ro_core import NDVIProcessor, Sentinel1FloodDetector, FireDetector, run_safe_ro_pipeline


ModuleNotFoundError: No module named 'safe_ro_core'

In [ ]:
# Synthetic data size
width, height = 200, 200

transform = from_origin(0, 0, 10, 10)  # arbitrary geotransform

def save_fake_raster(path, base_value, noise=5):
    data = base_value + noise * np.random.randn(height, width)
    with rasterio.open(
        path,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype="float32",
        crs="+proj=latlong",
        transform=transform,
    ) as dst:
        dst.write(data.astype("float32"), 1)

# Fake RED and NIR bands (simulate vegetation)
save_fake_raster("fake_red.tif", base_value=1000)
save_fake_raster("fake_nir.tif", base_value=1500)

# Fake Sentinel-1 band (low values = water)
s1_data = -15 + 3 * np.random.randn(height, width)
# add a "flooded" patch with lower backscatter
s1_data[50:120, 80:160] -= 5
with rasterio.open(
    "fake_s1.tif",
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype="float32",
    crs="+proj=latlong",
    transform=transform,
) as dst:
    dst.write(s1_data.astype("float32"), 1)

# Fake FIRMS CSV
fires_df = pd.DataFrame({
    "latitude": np.random.uniform(44, 47, size=50),
    "longitude": np.random.uniform(22, 28, size=50),
    "confidence": np.random.randint(50, 100, size=50),
})
fires_df.to_csv("fake_fires.csv", index=False)

"Files created: fake_red.tif, fake_nir.tif, fake_s1.tif, fake_fires.csv"


In [ ]:
ndvi_proc = NDVIProcessor("fake_red.tif", "fake_nir.tif")
ndvi = ndvi_proc.compute_ndvi()

print("NDVI stats:",
      "min =", float(ndvi.min()),
      "max =", float(ndvi.max()),
      "mean =", float(ndvi.mean()))

plt.imshow(ndvi, cmap="RdYlGn")
plt.colorbar(label="NDVI")
plt.title("Synthetic NDVI")
plt.show()


In [ ]:
flood_det = Sentinel1FloodDetector("fake_s1.tif")
mask = flood_det.detect()   # automatic threshold

flooded_percent = float(mask.mean() * 100.0)
print(f"Estimated flooded area: {flooded_percent:.2f}% of pixels")

plt.imshow(mask, cmap="Blues")
plt.title("Synthetic flood mask")
plt.colorbar()
plt.show()


In [ ]:
fire_det = FireDetector("fake_fires.csv")
high_conf_fires = fire_det.filter_by_confidence(80)

print("Number of fires with confidence ≥ 80:", len(high_conf_fires))
high_conf_fires.head()


In [ ]:
result = run_safe_ro_pipeline(
    s1_flood_path="fake_s1.tif",
    red_path="fake_red.tif",
    nir_path="fake_nir.tif",
    firms_csv="fake_fires.csv",
)

print("NDVI stats from pipeline:", result["ndvi_stats"])
print("Flooded area from pipeline: {:.2f}%".format(result["flooded_area_percent"]))
print("High-confidence fires from pipeline:", len(result["fires_high_conf"]))

plt.figure()
plt.imshow(result["ndvi"], cmap="RdYlGn")
plt.title("NDVI (from pipeline)")
plt.colorbar()
plt.show()

plt.figure()
plt.imshow(result["flood_mask"], cmap="Blues")
plt.title("Flood mask (from pipeline)")
plt.colorbar()
plt.show()
